## Calculate ion count for a specific amino acid type

In [ ]:
## calculate number of ions per AA type from CPPTRAJ contact mask output files (.dat output file type):

In [ ]:
# This function reads in and combines CPPTRAJ contact mask .dat files for each chain and then removes duplicates.

import pandas as pd
import os

def combine_and_sort_data(run_folder, file_folder):
    for data_type in file_folder: 
        for run_number in run_folder:
            print(f"Processing Run {run_number}")

            # Initialize an empty DataFrame to store data for this run
            combined_df = pd.DataFrame()

            # Loop through the number of protein chains in simulation (8 in this example)
            for chain_number in range(1, 9):
                folder_name = f"Run{run_number}/chain_data/ionMask_NtoCl_OtoNa_1500ns_chain{chain_number}"
                filename = os.path.join(folder_name, f"{data_type}.dat")  

                if os.path.exists(filename):
                    # Read the data from the current chain file
                    df = pd.read_csv(filename, delim_whitespace=True)

                    # Drop duplicates within the same Frame, keeping the first occurrence of each ResNum
                    df = df.drop_duplicates(subset=['#Frame', 'ResNum'], keep='first')

                    # Append the current chain's data to the combined DataFrame and repeat for each chain in the simulation
                    combined_df = combined_df.append(df, ignore_index=True)

            # Sort the combined DataFrame by the '#Frame' column
            combined_df.sort_values(by='#Frame', inplace=True)
            
            combined_df = combined_df.drop_duplicates(subset=['#Frame', 'ResNum'], keep='first')    
            
            # Save the combined and sorted DataFrame
            output_filename = f"Run{run_number}/chain_data/1st_shell_ion_per_res/allChainData_1500ns_{data_type}.csv"
            combined_df.to_csv(output_filename, index=False)

            print(f"Saved combined and sorted data for Run {run_number} to {output_filename}")


In [ ]:
## Run the following cells for each ion-amino_acid pair to remove duplicates from the raw data 

In [ ]:
#run_folders can be adjusted for any number of replicate simulations
run_folders = [1, 2, 3, 4] 
#file_folders is the file name of each .dat output file from CPPTRAJ
file_folders = ["Cl_within_4A_ARG", "Cl_within_4A_ASN", "Cl_within_4A_GLN", "Cl_within_4A_LYS", 
               "Cl_within_4A_SER", "Cl_within_4A_BB_N", "Cl_within_4A_GLY_BB_N"]
combine_and_sort_data(run_folders,file_folders)


In [ ]:
run_folders = [1, 2, 3, 4] 
file_folders = ["Na_within_3A_ASP", "Na_within_3A_ASN", "Na_within_3A_GLN", "Na_within_3A_SER", 
                "Na_within_3A_BB_O", "Na_within_3A_GLY_BB_O"]
combine_and_sort_data(run_folders,file_folders)


In [ ]:
#calculate average ions with each group after sorting data above
def calculate_average_sum(run_folders, file_folders, output_file, frames):
    result = []
    for data_type in file_folders:
        contact_sum = 0
        for run_folder in run_folders:
            filename = f"{run_folder}/chain_data/1st_shell_ion_per_res/allChainData_1500ns_{data_type}.csv"
            df = pd.read_csv(filename, sep='\t')
            contact_sum += len(df)
        avg_contact_sum = contact_sum / len(run_folders) / frames
        result.append((data_type, avg_contact_sum))
    
    # Save the result to a CSV file
    df_result = pd.DataFrame(result, columns=['AA', 'Ions'])
    df_result.to_csv(output_file, index=False)

In [ ]:
## Each "Run#" is a replicate simulation, for additional runs repeat the cells below 
## For reach Run#, run the 'calculate_average_sum' function for each ion type and include the amino acid file names to get average Cl-/Na+ ions for each amino_acid saved to one file


In [ ]:
run_folders = ['Run{}'.format(i) for i in [1]]
file_folders = ["Cl_within_4A_ARG", "Cl_within_4A_ASN", "Cl_within_4A_GLN", "Cl_within_4A_LYS", 
               "Cl_within_4A_SER", "Cl_within_4A_BB_N", "Cl_within_4A_GLY_BB_N"]
output_file = f"ion_data/Cl_within_4A_AA_run1.dat"
frames = 2500

calculate_average_sum(run_folders, file_folders, output_file, frames)

In [ ]:
run_folders = ['Run{}'.format(i) for i in [1]]
file_folders = ["Na_within_3A_ASP", "Na_within_3A_ASN", "Na_within_3A_GLN", "Na_within_3A_SER", 
                "Na_within_3A_BB_O", "Na_within_3A_GLY_BB_O"]
output_file = f"ion_data/Na_within_3A_AA_Run1.dat"
frames = 2500

calculate_average_sum(run_folders, file_folders, output_file, frames)

## Calculate number of ions bridging specific number of chains

In [1]:
## Calculate number of ions per chain number for bridging
# update 'i' variable as needed for more or less replicate simulations
# update 'frame_number' as needed for simulation frames in the input file
# update number/location of 'folder_names' based on number of chains in the simulation 
# update second variable in 'folder_path' based on the file name of the CPPTRAJ .dat ourput file for each ion-cutoff-protein selection
# 'output_names_NaOnly.txt' is a txt file containing the atom ID for each ion in the simulation (note: should match numbering for the CPPTRAJ .dat file that will be read in)

In [ ]:
import pandas as pd
import os

# Create an empty DataFrame to store the results
output_df = pd.DataFrame(columns=['i', 'Number', 'Value'])

# 'i' variable is for number of repliace runs (Run#) 
for i in range(1, 5):
    # 'output_names_NaOnly.txt' is a .txt file containing the atom ID for each Na+ ion in the simulation
    with open('output_names_NaOnly.txt', 'r') as resnum_file:
        unique_resnums = [line.strip() for line in resnum_file]

    frame_numbers = list(range(1, 2501))
    df_Na = pd.DataFrame(index=frame_numbers)

    for resnum in unique_resnums:
        df_Na[resnum] = 0
    
    #each folder contains the CPPTRAJ output files for contact mask calculaiton of ions within cutoffs to all protein heavy atoms
    folder_names = [f'Run{i}/chain_data/ionMask_NtoCl_OtoNa_bothCutoffs_1500ns_chain1/',
                    f'Run{i}/chain_data/ionMask_NtoCl_OtoNa_bothCutoffs_1500ns_chain2/',
                    f'Run{i}/chain_data/ionMask_NtoCl_OtoNa_bothCutoffs_1500ns_chain3/',
                    f'Run{i}/chain_data/ionMask_NtoCl_OtoNa_bothCutoffs_1500ns_chain4/',
                    f'Run{i}/chain_data/ionMask_NtoCl_OtoNa_bothCutoffs_1500ns_chain5/',
                    f'Run{i}/chain_data/ionMask_NtoCl_OtoNa_bothCutoffs_1500ns_chain6/',
                    f'Run{i}/chain_data/ionMask_NtoCl_OtoNa_bothCutoffs_1500ns_chain7/',
                    f'Run{i}/chain_data/ionMask_NtoCl_OtoNa_bothCutoffs_1500ns_chain8/',]  

    # .dat file in the folder path below is the file name for the output from cpptraj contact mask function for a given cutoff distance and protein atom selection, in this case its Na+ ions within 3A of protein heavy atoms
    for folder_name in folder_names:
        folder_path = os.path.join(folder_name, 'Na_within_3A_protAtom.dat')

        data_df = pd.read_csv(folder_path, sep='\s+', header=None, names=['Frame', 'AtomNum', 'Atom', 'ResNum', 'Res', 'MolNum'], skiprows=1)
        
        #counts number of times an ion shows up on the interaction list for a chain
        for index, row in data_df.iterrows():
            frame = int(row['Frame'])
            resnum = row['ResNum']
            df_Na.at[frame, f"{resnum}"] += 1
    #num variable represents the number of chains an ion can interact with, set high enough so that the largest chain number has 0 ions indicating the num-1 is maximum number of chains any ion interacts with at one time
    for num in range(1,7):
        result_list = []

        for index, row in df_Na.iterrows():
            sum_result = (row == num).sum().sum()  # Count occurrences of num in the row
            result_list.append(sum_result)

        result_df = pd.DataFrame({'Result': result_list})

        output_df = output_df.append({'i': i, 'Number': num, 'Value': result_df['Result'].mean()}, ignore_index=True)

#output file will contain 3 columns: the Run#, the number of bridged chains from 1 to max value 'num', followed by the number of unique ions interacting with that number of chains averged over the number of frames 
csv_file_path = 'ion_data/Na_3A_prot_1500ns.csv'
output_df.to_csv(csv_file_path, index=False)

print("Output saved to 'Na_3A_prot_1500ns.csv'")

